In [27]:
import pandas as pd
import sqlite3




## building database for Flask app

In [32]:
df = pd.read_csv("../static/data/slc_daily_pm2.5_pm10_2016to2024.csv")
df = df.sort_values(['date'], ascending = False)

# code for just latest
#latest_sensor_df = df.groupby(['sensor_id']).first().reset_index()
#latest_sensor_df = latest_sensor_df[['sensor_id','latitude','longitude','pm2.5_atm']]

In [33]:
# examine missing values for altitude
df_sensors = df[['sensor_id','latitude','longitude','date','altitude','pm2.5_atm','pm10.0_atm']]
df_sensors.rename(columns = {'pm2.5_atm':'pm2','pm10.0_atm':'pm10'},inplace=True)
df_sensors['date'] = pd.to_datetime(df['date'], format='%m/%d/%y')
sum(df_sensors['altitude'].isna())

/var/folders/kn/v0dr_8xn1qj15xgwlhdyfzph0000gn/T/ipykernel_82229/1837659829.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sensors.rename(columns = {'pm2.5_atm':'pm2','pm10.0_atm':'pm10'},inplace=True)
/var/folders/kn/v0dr_8xn1qj15xgwlhdyfzph0000gn/T/ipykernel_82229/1837659829.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sensors['date'] = pd.to_datetime(df['date'], format='%m/%d/%y')


9453

In [34]:
# Define the SQLite database name
database_name = '../static/data/sensors_readings_2016_present.db'

# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect(database_name)

# Convert the DataFrame to a SQL table within the database
# Replace 'data_table' with your desired table name
# Correct usage with a string table name
df_sensors.to_sql('sensors_readings', conn, if_exists='replace', index=False)


# Close the connection to the database
conn.close()

In [35]:
df_sensors

,sensor_id,latitude,longitude,date,altitude,pm2,pm10
213936,13273,40.707290,-112.033980,2023-09-09,4256.0,2.8870,3.2320
196089,525,40.783870,-111.870140,2023-09-09,4891.0,1.0380,1.2115
235838,42097,40.739105,-111.806370,2023-09-09,5178.0,2.4090,2.6345
198810,5408,40.987300,-111.928600,2023-09-09,4237.0,2.4250,2.6600
238107,45627,41.143090,-111.939280,2023-09-09,4491.0,1.7875,2.0360
...,...,...,...,...,...,...,...
2747,984,40.595387,-111.807755,2018-01-01,NaN,9.1195,10.4010
4339,5174,41.172134,-111.923890,2018-01-01,5188.0,5.6710,6.1615
2133,77,40.750816,-111.825290,2018-01-01,NaN,10.3680,11.3290
574,77,40.750816,-111.825290,2017-01-01,NaN,43.5200,56.6090
